# qiime2 v.2019.4 dunfield lab tutorial

* Relative paths are used
* All commands imply terminal is in the current working directory
* Knowledge of linux terminal is highly incouraged

## Activate qiime2 environment

In [ ]:
!conda activate qiime2-2019.4

## Optionally enable qiime-specific autocompletion

In [ ]:
!source tab-qiime

## Navigate to your working directory
* qiime2_lab_tutorial already folder contains raw data
* lab_pipeline folder contains trained NaiveBayes Classifier
* mapping file

Next step expects that the folder exists (example)

In [ ]:
!cd ~/Desktop/qiime2_lab_tutorial/lab_pipeline

## Let's view the directory content

In [1]:
!ls -l

total 64
-rwxrwxrwx 1 andriy andriy 55152 Oct  7 15:54 dunfield_lab_qiime2_pipeline_upd.ipynb
-rwx------ 1 andriy andriy   746 Oct 21 08:20 mappingfile.tsv
lrwxrwxrwx 1 andriy andriy   115 Oct 21 08:21 v3v4_silva132_classifier_wps2_2groups.qza -> /mnt/linuxData/qiimefiles/qiime2/classifierTraining/my_v3v4_with_a_script/v3v4_silva132_classifier_wps2_2groups.qza


## Trim primers

* Various primer trimming tools exist
    * cutadapt
    * bbmap
    * qiime2 native tools
    * trimmomatic
    * manual removal...

It is critical to remove non-biological sequences from the data.<br>
We will remove our 16S V3-V4 region (Bacteria-specific primer set) primers sequences using cutadapt <br>
* f-primer CCTACGGGNGGCWGCAG
* r-primer GACTACHVGGGTATCTAATCC

## define variables

In [27]:
classifier = "v3v4_silva132_classifier_wps2_2groups.qza"
metadata = "mappingfile.csv"


### Making directories

In [2]:
!mkdir primer_trimmed_fastqs; mkdir cutadapt_logs

### Primer trimming w\ cutadapt with a help of a little script :)
#### ! Expects to contain our data in the <font color=red>raw_data</font> folder in a parent directory
* cutadapt logs could be found ./primer_trimmed_fastqs/logs
* added minimum length parameter, since it gave an error with 0 length read

In [3]:
%%bash
for file1 in ../raw_data/*_R1_*.fastq.gz; do
    file2="${file1%_R1_001.fastq.gz}_R2_001.fastq.gz"
    fname1=`basename $file1`
    fname2=`basename $file2`
    `cutadapt --pair-filter any -j 4 -m 100 --no-indels --discard-untrimmed \
    -g CCTACGGGNGGCWGCAG -G GACTACHVGGGTATCTAATCC \
    -o primer_trimmed_fastqs/$fname1 -p primer_trimmed_fastqs/$fname2 \
    $file1 $file2 \
    > cutadapt_logs/${fname1}_cutadapt_log.txt`
done

## Import trimmed FASTQs as a QIIME2 artifact

To keep the directory clean you can put the artifact files in a new directory

In [4]:
!mkdir paired_reads_qza

### Casava 1.8 single-end demultiplexed fastq
Format description

In the Casava 1.8 demultiplexed (single-end) format, there is one fastq.gz file for each sample in the study which contains the single-end reads for that sample. The file name includes the sample identifier and should look like L2S357_15_L001_R1_001.fastq.gz. The underscore-separated fields in this file name are:

    the sample identifier,
    the barcode sequence or a barcode identifier,
    the lane number,
    the direction of the read (i.e. only R1, because these are single-end reads), and
    the set number.

Obtaining example data

### Importing...

In [5]:
!qiime tools import --type SampleData[PairedEndSequencesWithQuality] \
                   --input-path primer_trimmed_fastqs \
                   --output-path paired_reads_qza/reads_trimmed.qza \
                   --input-format CasavaOneEightSingleLanePerSampleDirFmt

Imported primer_trimmed_fastqs as CasavaOneEightSingleLanePerSampleDirFmt to paired_reads_qza/reads_trimmed.qza


* Our reads are now ready to be used by qiime2

## Quality control w/ deblur:
Currently deblur doesn't support paired-end reads <br>
### Using VSEARCH for joining:

In [6]:
!qiime vsearch join-pairs \
--i-demultiplexed-seqs paired_reads_qza/reads_trimmed.qza \
--o-joined-sequences paired_reads_qza/reads_trimmed_joined.qza

Saved SampleData[JoinedSequencesWithQuality] to: paired_reads_qza/reads_trimmed_joined.qza


### Filter out low-quality reads.

This command will filter out low-quality reads based on the default options.<br>
(this step may take a while)

In [7]:
!qiime quality-filter q-score-joined \
--i-demux paired_reads_qza/reads_trimmed_joined.qza \
--o-filter-stats filt_stats.qza \
--o-filtered-sequences paired_reads_qza/reads_trimmed_joined_filt.qza

Saved SampleData[JoinedSequencesWithQuality] to: paired_reads_qza/reads_trimmed_joined_filt.qza
Saved QualityFilterStats to: filt_stats.qza


### Deblur Workflow

This workflow is 16S sequences, for other amplicon regions, you can use the denoise-other option in the command and specify a reference database.

Note that you will need to trim all sequences to the same length with the --p-trim-length option. In order to determine the correct length to trim down to, run the following QC:

### To find appropriate deblur parameters we need to summarize our joined reads

In [8]:
!qiime demux summarize \
--i-data paired_reads_qza/reads_trimmed_joined_filt.qza \
--o-visualization reads_trimmed_joined_filt_summary.qzv

Saved Visualization to: reads_trimmed_joined_filt_summary.qzv


### View the obtained visualization

In [9]:
!qiime tools view reads_trimmed_joined_filt_summary.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.[22489:22509:1125/171926.327397:ERROR:browser_process_sub_thread.cc(203)] Waited 8 ms for network service
Opening in existing browser session.

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

### Qiime help on importing/exporting/viewing artefacts

In [17]:
#!qiime tools --help

### Explore provenance w/ https://view.qiime2.org

#### Showing on denoise-16S

In [18]:
#!qiime deblur denoise-16S --help

### Denoising w/ deblur
* Here I'm using the default behaviour of --p-min-reads = 5
* Changed min reads here 10-> to be more sensitive
* Reads are trimmed to 401nt which retains is at least 98% of the reads<br>
(this step may take a while depending on the size of your data ...)

In [10]:
!qiime deblur denoise-16S \
--i-demultiplexed-seqs paired_reads_qza/reads_trimmed_joined_filt.qza \
--p-trim-length 401 \
--p-sample-stats \
--p-jobs-to-start 7 \
--p-min-reads 5 \
--output-dir deblur_output

Saved FeatureTable[Frequency] to: deblur_output/table.qza
Saved FeatureData[Sequence] to: deblur_output/representative_sequences.qza
Saved DeblurStats to: deblur_output/stats.qza


### Output is saved in the deblur_output folder
#### let's summarise our deblur output

In [11]:
!qiime deblur visualize-stats \
  --i-deblur-stats deblur_output/stats.qza \
  --o-visualization deblur_output/deblur-stats.qzv

Saved Visualization to: deblur_output/deblur-stats.qzv


In [12]:
!qiime tools view deblur_output/deblur-stats.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.[26606:26625:1125/172405.338053:ERROR:browser_process_sub_thread.cc(203)] Waited 3 ms for network service
Opening in existing browser session.

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [13]:
!qiime feature-table summarize \
--i-table deblur_output/table.qza \
--o-visualization deblur_output/deblur_table_summary.qzv

Saved Visualization to: deblur_output/deblur_table_summary.qzv


In [14]:
!qiime tools view deblur_output/deblur_table_summary.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.[27159:27159:1125/172551.944131:ERROR:sandbox_linux.cc(369)] InitializeSandbox() called with multiple threads in process gpu-process.
[27121:27140:1125/172551.958422:ERROR:browser_process_sub_thread.cc(203)] Waited 3 ms for network service
Opening in existing browser session.

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

### Tabulate representative sequences

In [15]:
!qiime feature-table tabulate-seqs \
  --i-data deblur_output/representative_sequences.qza \
  --o-visualization representative_sequences.qzv

Saved Visualization to: representative_sequences.qzv


In [16]:
!qiime tools view representative_sequences.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.[27542:27561:1125/172647.434790:ERROR:browser_process_sub_thread.cc(203)] Waited 3 ms for network service
Opening in existing browser session.

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

## Building phylogeny with FastTree
### Making multiple-sequence alignment

We'll need to make a multiple-sequence alignment of the ASVs before running FastTree.

In [17]:
!mkdir fast_tree_out

In [18]:
!qiime alignment mafft \
--i-sequences deblur_output/representative_sequences.qza \
--p-n-threads 8 \
--o-alignment fast_tree_out/rep_seqs_mafft.qza


Saved FeatureData[AlignedSequence] to: fast_tree_out/rep_seqs_mafft.qza


### Filtering multiple-sequence alignment

Variable positions in the alignment need to be masked before FastTree is run, which can be done with this command:

In [19]:
!qiime alignment mask --i-alignment fast_tree_out/rep_seqs_mafft.qza \
  --o-masked-alignment fast_tree_out/rep_seqs_mafft_masked.qza

Saved FeatureData[AlignedSequence] to: fast_tree_out/rep_seqs_mafft_masked.qza


### Running FastTree

Finally FastTree can be run on this masked multiple-sequence alignment:

In [20]:
!qiime phylogeny fasttree \
--i-alignment fast_tree_out/rep_seqs_mafft_masked.qza \
--p-n-threads 4 \
--o-tree fast_tree_out/rep_seqs_aligned_masked_tree

Saved Phylogeny[Unrooted] to: fast_tree_out/rep_seqs_aligned_masked_tree.qza


### Make a rooted tree

Use midpoint root

In [21]:
!qiime phylogeny midpoint-root \
--i-tree fast_tree_out/rep_seqs_aligned_masked_tree.qza \
--o-rooted-tree fast_tree_out/rep_seqs_mafft_masked_tree_rooted.qza

Saved Phylogeny[Rooted] to: fast_tree_out/rep_seqs_mafft_masked_tree_rooted.qza


### Generate rarefaction curves

* Useful QC step
* Determine maximum depth for the rarefaction using following (I'm using 8000):


In [31]:
!qiime tools view deblur_output/deblur_table_summary.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.[2994:3013:1021/091330.986511:ERROR:browser_process_sub_thread.cc(203)] Waited 5 ms for network service
Opening in existing browser session.

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [32]:
#!qiime diversity alpha-rarefaction --help

In [22]:
!qiime diversity alpha-rarefaction \
--i-table deblur_output/table.qza \
--p-max-depth 15000 \
--p-metrics simpson \
--p-metrics faith_pd \
--p-metrics dominance \
--p-metrics chao1 \
--p-metrics observed_otus \
--p-metrics shannon \
--p-steps 20 \
--i-phylogeny fast_tree_out/rep_seqs_mafft_masked_tree_rooted.qza \
--o-visualization rarefaction_curves.qzv

Saved Visualization to: rarefaction_curves.qzv


In [23]:
!qiime tools view rarefaction_curves.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.[28603:28622:1125/173013.208917:ERROR:browser_process_sub_thread.cc(203)] Waited 3 ms for network service
Opening in existing browser session.

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

### Using metadata w\ rarefaction

In [29]:
!qiime diversity alpha-rarefaction \
--i-table deblur_output/table.qza \
--p-max-depth 15000 \
--p-steps 20 \
--i-phylogeny fast_tree_out/rep_seqs_mafft_masked_tree_rooted.qza \
--m-metadata-file $metadata \
--o-visualization rarefaction_metadata_curves.qzv

Saved Visualization to: rarefaction_metadata_curves.qzv


In [30]:
!qiime tools view rarefaction_metadata_curves.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.[29460:29479:1125/173418.547533:ERROR:browser_process_sub_thread.cc(203)] Waited 3 ms for network service
Opening in existing browser session.

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

### Filtering out samples if needed

In [124]:
# !echo SampleID > samples-to-exclude.tsv
# !echo nm1-9a >> samples-to-exclude.tsv
# !echo o1 >> samples-to-exclude.tsv
# !echo o29 >> samples-to-exclude.tsv
# !echo o20 >> samples-to-exclude.tsv
# !echo o7 >> samples-to-exclude.tsv

# !qiime feature-table filter-samples \
#   --p-exclude-ids \
#   --i-table deblur_output/table.qza \
#   --m-metadata-file samples-to-exclude.tsv \
#   --o-filtered-table id-filtered-deblur-table.qza


Saved FeatureTable[Frequency] to: id-filtered-deblur-table.qza


## Assign taxonomy
* Could be assigned to ASVs using a Naive-Bayes classifier
* This classifier was trained using SILVA 132 database and is specific for v3v4 region
* Contains edits for WPS-2 (Rubrimentifilales and AS-11)
* Could be trained <i>de novo</i>, but RAM intensive
* Qiime version sensitive

(this step may take a long time to complete ...)

In [31]:
!qiime feature-classifier classify-sklearn \
--i-reads deblur_output/representative_sequences.qza \
--i-classifier $classifier \
--output-dir taxonomy

Saved FeatureData[Taxonomy] to: taxonomy/classification.qza


### Our taxonomy folder now contains classification.qza file
let's explore the results..

#### Following command export the classification as a tsv-file


In [32]:
!qiime tools export --input-path taxonomy/classification.qza --output-path taxonomy

Exported taxonomy/classification.qza as TSVTaxonomyDirectoryFormat to directory taxonomy


### At last..., Our Beloved Bar-Chart :)

In [33]:
!qiime taxa barplot \
--i-table deblur_output/table.qza \
--i-taxonomy taxonomy/classification.qza \
--m-metadata-file $metadata \
--o-visualization taxonomy/taxa_barplot.qzv

Saved Visualization to: taxonomy/taxa_barplot.qzv


In [34]:
!qiime tools view taxonomy/taxa_barplot.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.[10032:10032:1125/184510.740856:ERROR:sandbox_linux.cc(369)] InitializeSandbox() called with multiple threads in process gpu-process.
[9981:10012:1125/184510.853767:ERROR:browser_process_sub_thread.cc(203)] Waited 3 ms for network service
Opening in existing browser session.

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

## Finally, let's calculate core diversity metrics
* For this step we need to select a reasonable rarefaction value
* Let's have a look at our FeatureTable again

#### 400 seems to be a good number in this case, we are loosing only 2 samples

In [37]:
!qiime diversity core-metrics-phylogenetic \
--i-phylogeny fast_tree_out/rep_seqs_mafft_masked_tree_rooted.qza \
--i-table deblur_output/table.qza \
--p-sampling-depth 2000 \
--m-metadata-file $metadata \
--output-dir core-metrics

Saved FeatureTable[Frequency] to: core-metrics/rarefied_table.qza
Saved SampleData[AlphaDiversity] % Properties('phylogenetic') to: core-metrics/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics/observed_otus_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics/evenness_vector.qza
Saved DistanceMatrix % Properties('phylogenetic') to: core-metrics/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix % Properties('phylogenetic') to: core-metrics/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics/jaccard_pcoa_results.qza
Saved PCoAResults to: core-metrics/bray_curtis_pcoa_results.qza
Saved Visua

#### let's view an ordination plot

In [38]:
!qiime tools view core-metrics/weighted_unifrac_emperor.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.[15892:15911:1125/191936.777783:ERROR:browser_process_sub_thread.cc(203)] Waited 7 ms for network service
Opening in existing browser session.

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

### Alpha diversity group significance test
* An example of just one metric

In [40]:
!qiime diversity alpha-group-significance \
--i-alpha-diversity core-metrics/faith_pd_vector.qza \
--m-metadata-file $metadata \
--o-visualization core-metrics/faith-pd-group-significance.qzv

Saved Visualization to: core-metrics/faith-pd-group-significance.qzv


In [41]:
!qiime tools view core-metrics/faith-pd-group-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.[16421:16440:1125/192100.919168:ERROR:browser_process_sub_thread.cc(203)] Waited 3 ms for network service
Opening in existing browser session.

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

### Beta diversity group significance test
* lets test weighted unifrac

In [43]:
!qiime diversity beta-group-significance \
--i-distance-matrix core-metrics/weighted_unifrac_distance_matrix.qza \
--m-metadata-file $metadata \
--m-metadata-column Source \
--p-pairwise \
--o-visualization core-metrics/unweighted-unifrac-bodysite-significance.qzv

Saved Visualization to: core-metrics/unweighted-unifrac-bodysite-significance.qzv


In [44]:
!qiime tools view core-metrics/unweighted-unifrac-bodysite-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.[16981:17001:1125/192242.002044:ERROR:browser_process_sub_thread.cc(203)] Waited 3 ms for network service
Opening in existing browser session.

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

## Bonus part: Exporting FeatureTables (biom files)
* qiime2 keeps taxonomy separately
* therefore exporting biom files with taxonomy needs some additional steps

In [45]:
!sed -i -e '1 s/Feature/#OTUID/' -e '1 s/Taxon/taxonomy/' taxonomy/taxonomy.tsv

In [46]:
!qiime tools export \
--input-path deblur_output/table.qza \
--output-path deblur-table-exported

Exported deblur_output/table.qza as BIOMV210DirFmt to directory deblur-table-exported


In [47]:
!biom add-metadata \
-i deblur-table-exported/feature-table.biom \
-o deblur-table-exported/feature-table_w_tax.biom \
--observation-metadata-fp taxonomy/taxonomy.tsv \
--sc-separated taxonomy

### And finally a familiar biom-convert :)

In [48]:
!biom convert \
-i deblur-table-exported/feature-table_w_tax.biom \
-o deblur-table-exported/feature-table.tsv \
--to-tsv --header-key taxonomy

## Exporting w\ fractions

In [49]:
!qiime feature-table relative-frequency \
--i-table deblur_output/table.qza \
--o-relative-frequency-table frac-deblur-table.qza

Saved FeatureTable[RelativeFrequency] to: frac-deblur-table.qza


In [50]:
!qiime tools export --input-path frac-deblur-table.qza --output-path frac-deblur-table

Exported frac-deblur-table.qza as BIOMV210DirFmt to directory frac-deblur-table


In [51]:
!biom add-metadata \
-i frac-deblur-table/feature-table.biom \
-o frac-deblur-table/feature-table_w_tax.biom \
--observation-metadata-fp taxonomy/taxonomy.tsv --sc-separated taxonomy

In [52]:
!biom convert \
-i frac-deblur-table/feature-table_w_tax.biom \
-o frac-deblur-table/feature-table.tsv \
--to-tsv \
--header-key taxonomy

## Making biom tables with fraciton by taxonomic level

In [19]:
!mkdir taxa-levels

## Level 2

In [20]:
!qiime taxa collapse \
--i-table deblur_output/table.qza \
--i-taxonomy taxonomy/classification.qza \
--p-level 2 \
--o-collapsed-table taxa-levels/table-l2.qza

Saved FeatureTable[Frequency] to: taxa-levels/table-l2.qza


### convert counts to fractions

In [21]:
!qiime feature-table relative-frequency \
--i-table taxa-levels/table-l2.qza \
--o-relative-frequency-table taxa-levels/frac-table-l2.qza

Saved FeatureTable[RelativeFrequency] to: taxa-levels/frac-table-l2.qza


### Convert to tsv w\ taxonomy

In [22]:
!qiime tools export \
--input-path taxa-levels/frac-table-l2.qza \
--output-path taxa-levels/frac-table-l2

Exported taxa-levels/frac-table-l2.qza as BIOMV210DirFmt to directory taxa-levels/frac-table-l2


In [23]:
!biom convert \
-i taxa-levels/frac-table-l2/feature-table.biom \
-o taxa-levels/frac-table-l2/feature-table.tsv \
--to-tsv


## Level 4

In [24]:
!qiime taxa collapse \
--i-table deblur_output/table.qza \
--i-taxonomy taxonomy/classification.qza \
--p-level 4 \
--o-collapsed-table taxa-levels/table-l4.qza

!qiime feature-table relative-frequency \
--i-table taxa-levels/table-l4.qza \
--o-relative-frequency-table taxa-levels/frac-table-l4.qza

!qiime tools export \
--input-path taxa-levels/frac-table-l4.qza \
--output-path taxa-levels/frac-table-l4

!biom convert \
-i taxa-levels/frac-table-l4/feature-table.biom \
-o taxa-levels/frac-table-l4/feature-table.tsv \
--to-tsv


Saved FeatureTable[Frequency] to: taxa-levels/table-l4.qza
Saved FeatureTable[RelativeFrequency] to: taxa-levels/frac-table-l4.qza
Exported taxa-levels/frac-table-l4.qza as BIOMV210DirFmt to directory taxa-levels/frac-table-l4


## Good-bye!